In [5]:
from googletrans import Translator
from afinn import Afinn
import pandas as pd

In [6]:
news = pd.read_excel('../data/sentiment_base.xlsx')

In [5]:
news.head()

,text_body
0,«Электрокабель» приобрел компьютеры для Кольчу...
1,Спорт-лидер - это круто!\nКоммунар - Kommunar9...
2,Новый и очень редкий экспонат появился в колле...
3,Вести. Урал Новый и очень редкий экспонат появ...
4,НОВЫЙ И ОЧЕНЬ РЕДКИЙ ЭКСПОНАТ ПОЯВИЛСЯ В КОЛЛЕ...


In [5]:
translator = Translator(service_urls=['translate.googleapis.com'])
afinn = Afinn()

In [8]:
def text_analizer(news_text):
    
    def text_splitter(text):
        list_of_news = []
        itterations = len(text)//3000
        samle_size = len(text)
        sample_tail = len(text)%3000
        
        for i in range (0, itterations):
            str_to_remove = text[:3000]
            list_of_news.append(str_to_remove)
            text.replace(str_to_remove,'')
        list_of_news.append(text[samle_size-sample_tail:samle_size])
        
        return list_of_news
    
    def text_translator(list_of_news):
        news_en = []
        for i in list_of_news:
            news_en.append(translator.translate(i, lang_tgt='en').text)
        return news_en
    
    def text_scoring(news_en):
        news_string = ' '.join(news_en)
        return afinn.score(news_string)
    
    first_step = text_splitter(news_text)
    second_step = text_translator(first_step)
    final_step = text_scoring(second_step)
    
    return final_step

In [9]:
news['score'] = news['text_body'].apply(text_analizer)

In [10]:
news.head()

,text_body,score
0,«Электрокабель» приобрел компьютеры для Кольчу...,8.0
1,Спорт-лидер - это круто!\nКоммунар - Kommunar9...,11.0
2,Новый и очень редкий экспонат появился в колле...,18.0
3,Вести. Урал Новый и очень редкий экспонат появ...,3.0
4,НОВЫЙ И ОЧЕНЬ РЕДКИЙ ЭКСПОНАТ ПОЯВИЛСЯ В КОЛЛЕ...,29.0


In [11]:
with pd.ExcelWriter(
            "./data/sentiment_score.xlsx") as writer:
        news.to_excel(writer, sheet_name='score')

---

In [6]:
news['text_body'][1]

'Спорт-лидер - это круто!\nКоммунар - Kommunar96.ru (Туринская Слобода) , 2.1.2023\nВо Дворце спорта Уральской горно-металлургической компании города Верхняя Пышма прошло торжественное мероприятие по подведению итогов рейтинга. В течение года для спорт-лидеров была проведена серия мероприятий в социальных сетях в рамках реализации инновационного проекта «Спорт-лидер» национального проекта «Демография».\nПо итогам участия был сформирован рейтинг и определены десять лучших спорт-лидеров, набравших наибольшее количество баллов.\nВ десятку лучших попал и наш земляк - Ярослав Спицын из Слободо-Туринской школы 1. Его наставником является Алевтина Николаевна Волобуева (на снимке).\nВ рамках мероприятия все участники посетили планетарий на базе музея военной техники УГМК, баскетбольный матч во Дворце спорта УГМК и в торжественной обстановке были награждены Дипломами и Благодарственными письмами.\n'

In [9]:
# the following installations are required
# python -m textblob.download_corpora
# python -m spacy download en_core_web_sm
# ru_core_news_lg

import spacy
from spacytextblob.spacytextblob import SpacyTextBlob

nlp = spacy.load('ru_core_news_lg')
nlp.add_pipe('spacytextblob')
text = news['text_body'][10]
doc = nlp(text)
doc._.blob.polarity                            # Polarity: -0.125
#doc._.blob.subjectivity                        # Subjectivity: 0.9
#doc._.blob.sentiment_assessments.assessments   # Assessments: [(['really', 'horrible'], -1.0, 1.0, None), (['worst', '!'], -1.0, 1.0, None), (['really', 'good'], 0.7, 0.6000000000000001, None), (['happy'], 0.8, 1.0, None)]
#doc._.blob.ngrams()                            # [WordList(['I', 'had', 'a']), WordList(['had', 'a', 'really']), WordList(['a', 'really', 'horrible']), WordList(['really', 'horrible', 'day']), WordList(['horrible', 'day', 'It']), WordList(['day', 'It', 'was']), WordList(['It', 'was', 'the']), WordList(['was', 'the', 'worst']), WordList(['the', 'worst', 'day']), WordList(['worst', 'day', 'ever']), WordList(['day', 'ever', 'But']), WordList(['ever', 'But', 'every']), WordList(['But', 'every', 'now']), WordList(['every', 'now', 'and']), WordList(['now', 'and', 'then']), WordList(['and', 'then', 'I']), WordList(['then', 'I', 'have']), WordList(['I', 'have', 'a']), WordList(['have', 'a', 'really']), WordList(['a', 'really', 'good']), WordList(['really', 'good', 'day']), WordList(['good', 'day', 'that']), WordList(['day', 'that', 'makes']), WordList(['that', 'makes', 'me']), WordList(['makes', 'me', 'happy'])]


0.0